In [ ]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.colors as colors

In [ ]:
# Specify paths
parcel_file_csv = "Data/Beaufort_parcels_regression.csv"
parcel_file_shp = "Data/Beaufort_final_8.shp"
results_path = "Results/"  # Change this for different output files

### Buyers, sellers, transactions


In [ ]:
df_list = []

i = 0
for file in os.listdir(results_path):
    if file.startswith("model"):
        model_df = pd.read_pickle(results_path + file)
        model_df["Step"] = model_df.index
        model_df["Run"] = int(file.split("]")[0][-1])
        df_list.append(model_df)
        i += 1

model_df = pd.concat(df_list)
model_df["Year"] = 0.5 * model_df["Step"]
model_df

In [ ]:
%matplotlib inline
model_df_mean = model_df.groupby(["Year"])[["N_sellers", "N_buyers"]].mean()
model_df_std = model_df.groupby(["Year"])[["N_sellers", "N_buyers"]].std()

model_df_mean = model_df_mean[1:]
model_df_std = model_df_std[1:]

model_df_mean.plot()
for i, group in enumerate(["N_sellers", "N_buyers"]):
    plt.fill_between(model_df_mean.index,
                     (model_df_mean - model_df_std)[group],
                     (model_df_mean + model_df_std)[group],
                     color=plt.legend().legendHandles[i].get_color(),
                     alpha=0.5)

plt.ylabel("N agents")
plt.ylim(0, 1000)
plt.show()

### Prices, incomes, transactions per flood zone

In [ ]:
# Read spatial data
spatial_df = gpd.read_file(parcel_file_shp)
spatial_df = spatial_df.rename(columns={"ID1": "Property ID"}).set_index("Property ID")

# Redo original preprocessing for relevant parcel attributes
spatial_df.loc[spatial_df["DFLOOD_A_1"] == 1, "FLOOD_PROB"] = 0.01
spatial_df.loc[spatial_df["DFLOOD_X_1"] == 1, "FLOOD_PROB"] = 0.002
spatial_df["FLOOD_PROB"] = spatial_df["FLOOD_PROB"].drop(columns=["DFLOOD_A_1", "DFLOOD_X_1"])
spatial_df["FLOOD_PROB"] = spatial_df["FLOOD_PROB"].fillna(0)

# Disregard roads for visualization
spatial_df = spatial_df.drop([2, 4, 153, 923, 966, 1042, 1283, 1290, 1492, 1710, 1808, 2241, 2313,
                              2375, 2671, 2902, 3015, 3078, 3107, 3131, 3134, 3709, 3870, 4257, 4281, 5507])
# spatial_df.head()

In [ ]:
buyers_df_list = []
owners_df_list = []
i = 0
for file in os.listdir(results_path):
    if file.startswith("agent"):
        agent_df = pd.read_pickle(results_path + file)
        buyers_df = agent_df[(agent_df["Type"] == "Household") & (agent_df['Property ID'].isna())].copy()
        buyers_df["Run"] = int(file.split("]")[0][-1])
        buyers_df_list.append(buyers_df)
        owners_df = agent_df[(agent_df["Type"] == "Household") & (~agent_df['Property ID'].isna())].copy()
        owners_df["Run"] = int(file.split("]")[0][-1])
        owners_df_list.append(owners_df)
        i += 1

owners_df = pd.concat(owners_df_list)
owners_df = owners_df.reset_index().set_index("Property ID")
owners_df["FLOOD_PROB"] = spatial_df["FLOOD_PROB"]
owners_df["Year"] = 0.5 * owners_df["Step"]
owners_df["geometry"] = spatial_df["geometry"]
owners_df = gpd.GeoDataFrame(owners_df)
owners_df = owners_df.reset_index().set_index(["Year", "Run", "FLOOD_PROB"])

buyers_df = pd.concat(buyers_df_list).reset_index()
buyers_df["Year"] = 0.5 * buyers_df["Step"]
buyers_df = buyers_df.set_index(["Year", "Run"])

owners_df

#### Household incomes

In [ ]:
# Buyers income
buyers_mean = buyers_df[["Income"]].groupby(["Year", "Run"]).mean().groupby("Year").mean()
buyers_std = buyers_df[["Income"]].groupby(["Year", "Run"]).mean().groupby("Year").std()

# Owners income per flood zone
owners_mean = owners_df[["Income"]].groupby(["Year", "Run", "FLOOD_PROB"]).mean()["Income"].groupby(["Year", "FLOOD_PROB"]).mean().unstack()
owners_std = owners_df[["Income"]].groupby(["Year", "Run", "FLOOD_PROB"]).mean()["Income"].groupby(["Year", "FLOOD_PROB"]).std().unstack()

# Do not plot first timestep
owners_mean = owners_mean.loc[1:]
owners_std = owners_std.loc[1:]
buyers_mean = buyers_mean[1:]
buyers_std = buyers_std[1:]

# Plot average incomes + std
fig, ax = plt.subplots()
owners_mean.plot(ax=ax)
for i, prob in enumerate([0, 0.002, 0.01]):
    ax.fill_between(owners_mean.index,
                    owners_mean[prob] - owners_std[prob],
                    owners_mean[prob] + owners_std[prob],
                    alpha=0.2, color=plt.legend().legendHandles[i].get_color())

ax.plot(buyers_mean, label="Buyers", color="tab:red", ls="--")
ax.fill_between(buyers_mean.index,
                (buyers_mean - buyers_std).values.flatten(),
                (buyers_mean + buyers_std).values.flatten(),
                alpha=0.2, color="tab:red")

plt.ticklabel_format(axis='y', scilimits=(0, 0))
plt.ylim(3e4, 1e5)
plt.ylabel("Average income (US Dollars)")
plt.title("Average income")
plt.legend(title="Flood zones")
plt.show()

#### Property prices

In [ ]:
%matplotlib inline
# Average property prices
owners_mean = owners_df[["Property price"]].groupby(["Year", "Run", "FLOOD_PROB"]).mean()["Property price"].groupby(["Year", "FLOOD_PROB"]).mean().unstack()
owners_std = owners_df[["Property price"]].groupby(["Year", "Run", "FLOOD_PROB"]).mean()["Property price"].groupby(["Year", "FLOOD_PROB"]).std().unstack()

owners_mean = owners_mean[1:]
owners_std = owners_std[1:]

fig, ax = plt.subplots()
owners_mean.plot(ax=ax)
for i, prob in enumerate([0, 0.002, 0.01]):
    ax.fill_between(owners_mean.index,
                    owners_mean[prob] - owners_std[prob],
                    owners_mean[prob] + owners_std[prob],
                    alpha=0.2, color=plt.legend().legendHandles[i].get_color())

plt.ticklabel_format(axis='y', scilimits=(0, 0))
plt.ylim(1.5e5, 5e5)
plt.ylabel("Average property price (US Dollars)")
plt.title("Average property price")
plt.legend(title="Flood zones")
plt.show()

#### Income/price ratio (compare runs)

In [ ]:
# Get ratio of property price / income / 30 (years mortgage).
# The resulting number is related to the maximum 30% of income spent on housing
# rule as described in Filatova (2015). The grey line indicated this maximum value.
ratio_df = owners_df[["Income", "Property price"]].copy()
ratio_df["Income price ratio"] = ratio_df["Property price"]/ratio_df["Income"]/30
ratio_df = ratio_df.groupby(["Year", "Run"]).mean()["Income price ratio"].unstack()
ratio_df.plot(cmap="tab10_r", alpha=0.8)
plt.hlines(0.3, min(ratio_df.index), max(ratio_df.index),
           color="grey", linestyle="--")
plt.show()

#### Income/price ratio (average over runs, per flood zone)

In [ ]:
# Owners income per flood zone
owners_df["Income price ratio"] = owners_df["Property price"]/owners_df["Income"]/30
owners_mean = owners_df[["Income price ratio"]].groupby(["Year", "Run", "FLOOD_PROB"]).mean()["Income price ratio"].groupby(["Year", "FLOOD_PROB"]).mean().unstack()
owners_std = owners_df[["Income price ratio"]].groupby(["Year", "Run", "FLOOD_PROB"]).mean()["Income price ratio"].groupby(["Year", "FLOOD_PROB"]).std().unstack()

# Plot average incomes + std
fig, ax = plt.subplots()
owners_mean.plot(ax=ax)
for i, prob in enumerate([0, 0.002, 0.01]):
    ax.fill_between(owners_mean.index,
                    owners_mean[prob] - owners_std[prob],
                    owners_mean[prob] + owners_std[prob],
                    alpha=0.2, color=plt.legend().legendHandles[i].get_color())

plt.hlines(0.3, min(ratio_df.index), max(ratio_df.index),
           color="grey", linestyle="--")
plt.ylabel("Ratio price/income")
plt.legend(title="Flood zones")
plt.show()

#### Transactions per flood zone

In [ ]:
# Load transactions and order per run per property
trans_per_flood_df = model_df[["Sold properties", "P_ask", "P_bid", "P_trans", "Run"]].copy()
trans_per_flood_df = trans_per_flood_df.drop(0)
trans_per_flood_df.index.rename("Year", inplace=True)
trans_per_flood_df.index = trans_per_flood_df.index * 0.5
trans_per_flood_df = trans_per_flood_df.set_index("Run", append=True)
trans_per_flood_df = trans_per_flood_df.explode(["Sold properties", "P_ask", "P_bid", "P_trans"])
trans_per_flood_df = trans_per_flood_df.reset_index().set_index(["Run", "Sold properties"])

trans_per_flood_df = pd.get_dummies(trans_per_flood_df["Year"])
trans_per_flood_df = trans_per_flood_df.groupby(["Run", "Sold properties"]).sum()
trans_per_flood_df = trans_per_flood_df.reset_index().set_index(["Sold properties"])
trans_per_flood_df = trans_per_flood_df.merge(spatial_df[["FLOOD_PROB"]], left_index=True, right_index=True)
trans_per_flood_df = trans_per_flood_df.groupby(["FLOOD_PROB", "Run"]).sum()
trans_per_flood_df = trans_per_flood_df.transpose()
trans_per_flood_df[0.01] = trans_per_flood_df[0.01]/944
trans_per_flood_df[0.002] = trans_per_flood_df[0.002]/832
trans_per_flood_df[0] = trans_per_flood_df[0]/1745

# Get mean and std over runs
trans_per_flood_df_mean = trans_per_flood_df.transpose().groupby("FLOOD_PROB").mean()
trans_per_flood_df_std = trans_per_flood_df.transpose().groupby("FLOOD_PROB").std()
trans_per_flood_df_std

trans_per_flood_df_mean.transpose().plot()
for i, prob in enumerate([0,  0.002, 0.01]):
    plt.fill_between(list(trans_per_flood_df_mean.columns.values),
                     trans_per_flood_df_mean.loc[prob] - trans_per_flood_df_std.loc[prob],
                     trans_per_flood_df_mean.loc[prob] + trans_per_flood_df_std.loc[prob],
                     alpha=0.2, color=plt.legend().legendHandles[i].get_color())

plt.ylim(0, 0.15)
plt.xlabel("Year")
plt.ylabel("Average fraction of properties sold")
plt.legend(title="Flood zone")
plt.show()

## Price estimation: fitting regression or kriging model

#### Regression coefficients

In [ ]:
%matplotlib inline
coefs_avg = model_df["Regression coefs"].groupby(model_df["Step"]).mean()
coefs_std = model_df[["Regression coefs"]].stack().groupby(level=0).apply(lambda x:
                                                                          np.round(np.array(x).std(), 4))

coefs_avg = pd.DataFrame(coefs_avg.values.tolist())
coefs_std = pd.DataFrame(coefs_std.values.tolist())

coefs_avg[0].plot(label="Intercept")
plt.fill_between(coefs_avg.index,
                 coefs_avg[0] - coefs_std[0],
                 coefs_avg[0] + coefs_std[0],
                 alpha=0.5)
# plt.ylim(10, 14)
plt.legend()
plt.show()

plt.rcParams["axes.prop_cycle"] = plt.cycler("color", plt.cm.tab10.colors)
if "Results/Kriging" in results_path:
    labels = ["Age", "House size", "Lot size", "N bedrooms", "Flood zone"]
else:
    labels = ["Bathroom", "Bathroom^2",
              "Age", "Age^2",
              "House size", "House size^2",
              "Lot size", "Lot size^2",
              "New home", "Post firm", "100 flood zone", "500 flood zone", "Coastal front",
              "Distance amen", "Distance CBD", "Distance highway", "Distance park"
             ]
coefs_avg = coefs_avg.drop(columns=0)
coefs_std = coefs_std.drop(columns=0)
for coef in coefs_avg.columns:
    plt.plot(coefs_avg[coef], label=labels[coef-1])
for coef in coefs_avg.columns:
    plt.fill_between(coefs_avg.index,
                     coefs_avg[coef] - coefs_std[coef],
                     coefs_avg[coef] + coefs_std[coef],
                     alpha=0.5)
plt.ylim(-0.5, 0.5)
plt.legend(loc=(0.98, 0))
plt.show()

#### Number of historical transactions needed to fit model


In [ ]:
%matplotlib inline

trans_history = model_df.groupby(["Year", "Run"])["Trans history"].mean().unstack()
trans_history.plot(cmap="tab20")
plt.ylabel("Length of history (N timesteps)")
plt.show()

## Spatial plots


#### Variables per agent/property

In [ ]:
property_df_list = []
i = 0
for file in os.listdir(results_path):
    if file.startswith("agent"):
        agent_df = pd.read_pickle(results_path + file)
        property_df = agent_df[(agent_df["Type"] == "Household") & (~agent_df['Property ID'].isna())].copy()
        property_df["Run"] = int(file.split("]")[0][-1])
        property_df_list.append(property_df)
        i += 1

# Add flood zones
property_df = pd.concat(property_df_list)
property_df = property_df.reset_index().set_index("Property ID")
property_df["FLOOD_PROB"] = spatial_df["FLOOD_PROB"]
property_df["Year"] = 0.5 * property_df["Step"]
property_df["geometry"] = spatial_df["geometry"]
property_df = gpd.GeoDataFrame(property_df)
property_df = property_df.reset_index().set_index(["Run", "Step", "Property ID"])
# property_df

In [ ]:
# Add price changes per property
price_gradients = property_df.sort_values(["Run", "Property ID"])["Property price"].diff()
property_df["Price change"] = price_gradients
property_df = property_df.reset_index().set_index("Property ID")
property_df.loc[property_df["Step"] == 0, ["Price change"]] = 0
# property_df

In [ ]:
# Add cumulative price changes
property_df = property_df.sort_values(["Run", "Property ID"])
prices = property_df.groupby(["Run", "Property ID"])["Price change"]
property_df["Cumulative price change"] = prices.cumsum()
property_df = property_df.reset_index().set_index("Property ID")
property_df

In [ ]:
def plot_beaufort(variable, year, logscale=False, divergent=False, save=False):
    fig, ax = plt.subplots(figsize=(8,8))
    ax.axis("off")

    # Plot parcels
    spatial_df[spatial_df["CITY"] != "Water"].plot(ax=ax,
                                                   color="sandybrown", alpha=0.4)
    spatial_df[spatial_df["CITY"] == "Water"].plot(ax=ax,
                                                   color="lightblue", alpha=0.7)

    # Plot variable of interest (averaged over runs)
    df = gpd.GeoDataFrame(property_df[property_df["Year"] == year].groupby(["Property ID"])[[variable]].mean())
    df["geometry"] = property_df[(property_df["Run"] == 0) & (property_df["Year"] == year)]["geometry"]
    if logscale:
        df.plot(variable, ax=ax,
                cmap="YlGn",
                legend=True, legend_kwds={"shrink": 0.3, "label": variable},
                norm=colors.LogNorm(vmin=1e5, vmax=1e7)
                )
    elif divergent:
        df.plot(variable, ax=ax,
                cmap="RdYlGn",
                legend=True, legend_kwds={"shrink": 0.3, "label": variable},
#                 norm=colors.TwoSlopeNorm(vmin=df[variable].min(), vcenter=0, vmax=df[variable].max()),
                vmin=-1e6,
                vmax=1e6
               )
    else:
        df.plot(variable, ax=ax,
                cmap="YlGn",
                legend=True, legend_kwds={"shrink": 0.3, "label": variable},
                vmin=1e2,
                vmax=1e5
               )

    # Plot flood zones
    flood_df = spatial_df.copy()
    flood_df["FLOOD_PROB"] = flood_df["FLOOD_PROB"].replace({0: np.nan})
    flood_df.plot("FLOOD_PROB", ax=ax,
                   categorical=True,
                   legend=True, legend_kwds={"title": "Flood probability"},
                   cmap="Blues",
                   norm=colors.TwoSlopeNorm(vmin=-1, vcenter=0, vmax=1.5),
                   alpha=0.5,
                   hatch=r"\\", edgecolor="tab:blue",
                   linewidth=0
                  )
    
    # Plot settings
    for ax in fig.axes:
        ax.tick_params(labelsize=8)
    plt.tight_layout()
    if save:
        plt.savefig("Figures/temp/Beaufort_" + variable + "_T" + str(year) + ".png",
                    dpi=300)
        plt.close()
    else:
        plt.show()

### Cumulative price changes

In [ ]:
%matplotlib notebook
plot_beaufort(variable="Cumulative price change", year=30, divergent=True)

In [ ]:
# %matplotlib inline
# for year in np.arange(0.5, 30, 1):
#     # NOTE: 30 years too long for plotting every year, but once in two years gives very sudden jumps.
#     plot_beaufort(variable="Cumulative price change", year=year, divergent=True, save=True)

### Household income

In [ ]:
%matplotlib notebook
plot_beaufort(variable="Income", year=1)
plot_beaufort(variable="Income", year=30)

In [ ]:
%matplotlib inline
for year in np.arange(0.5, 10, 1):
    plot_beaufort(variable="Income", year=year, logscale=False, save=True)

### Spatial distributions

#### Amenities

In [ ]:
parcel_df = pd.read_csv(parcel_file_csv)
parcel_df = parcel_df.rename(columns={"ID1": "Property ID"}).set_index("Property ID")

In [ ]:
temp = spatial_df[spatial_df.index.isin(property_df.index)].copy()
temp["PROXAMEN_1"] = parcel_df["PROXAMEN_1"]
print("Proximity coastal amenities not in flood zone:", temp[(temp["DFLOOD_X_1"] == 0) &
                                                             (temp["DFLOOD_A_1"] == 0)]["PROXAMEN_1"].mean())
print("Proximity coastal amenities for 500 flood zone:", temp[temp["DFLOOD_X_1"] == 1]["PROXAMEN_1"].mean())
print("Proximity coastal amenities for 100 flood zone:", temp[temp["DFLOOD_A_1"] == 1]["PROXAMEN_1"].mean())

In [ ]:
%matplotlib inline

fig, ax = plt.subplots(figsize=(8,8))
ax.axis("off")

# Plot parcels
spatial_df[spatial_df["CITY"] != "Water"].plot(ax=ax,
                                               color="peru", alpha=0.4)
spatial_df[spatial_df["CITY"] == "Water"].plot(ax=ax,
                                               color="lightblue", alpha=0.7)

# Plot distance to coastal amenities
temp = spatial_df[spatial_df.index.isin(property_df.index)].copy()
temp["PROXAMEN_1"] = parcel_df["PROXAMEN_1"]
temp.plot("PROXAMEN_1", ax=ax, cmap="Greys",
          legend=True, legend_kwds={"shrink": 0.3, "label": "Proximity coastal amenities"})

# Plot settings
for ax in fig.axes:
    ax.tick_params(labelsize=8)
plt.tight_layout()
plt.show()

#### Tax values

In [ ]:
print("Tax value not in flood zone:", spatial_df[(spatial_df["DFLOOD_X_1"] == 0) & (spatial_df["DFLOOD_A_1"] == 0)]["TAX_VALUE"].mean())
print("Tax value for 500 flood zone:", spatial_df[spatial_df["DFLOOD_X_1"] == 1]["TAX_VALUE"].mean())
print("Tax value for 100 flood zone:", spatial_df[spatial_df["DFLOOD_A_1"] == 1]["TAX_VALUE"].mean())

In [ ]:
%matplotlib notebook

fig, ax = plt.subplots(figsize=(8,8))
ax.axis("off")

# Plot parcels
spatial_df[spatial_df["CITY"] != "Water"].plot(ax=ax,
                                               color="peru", alpha=0.4)
spatial_df[spatial_df["CITY"] == "Water"].plot(ax=ax,
                                               color="lightblue", alpha=0.7)

# Plot distance to coastal amenities
temp = spatial_df[spatial_df.index.isin(property_df.index)].copy()
temp.plot("TAX_VALUE", ax=ax, cmap="Greys",
          norm=colors.LogNorm(vmin=1e4, vmax=1e6),
          legend=True, legend_kwds={"shrink": 0.3, "label": "Tax value"})

# Plot settings
for ax in fig.axes:
    ax.tick_params(labelsize=8)
plt.tight_layout()
plt.show()